In [1]:
# import dependencies
from pyspark.sql import SparkSession # needed to use spark DataFarmes
spark = SparkSession.builder.appName("recommender").getOrCreate()
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
# import data
df = spark.read.csv('/FileStore/tables/ratings.csv', inferSchema=True, header=True)

In [3]:
# explore data columns
df.printSchema()

root
-- userId: integer (nullable = true)
-- movieId: integer (nullable = true)
-- rating: double (nullable = true)
-- timestamp: integer (nullable = true)

In [4]:
# un-select timestamp column
df = df.select(['userId', 'movieId', 'rating'])

In [5]:
df.head(5)

Out[ 9 ]: 
[Row(userId=1, movieId=110, rating=1.0),
 Row(userId=1, movieId=147, rating=4.5),
 Row(userId=1, movieId=858, rating=5.0),
 Row(userId=1, movieId=1221, rating=5.0),
 Row(userId=1, movieId=1246, rating=5.0)]

In [6]:
df.show()

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 1| 110| 1.0|
 1| 147| 4.5|
 1| 858| 5.0|
 1| 1221| 5.0|
 1| 1246| 5.0|
 1| 1968| 4.0|
 1| 2762| 4.5|
 1| 2918| 5.0|
 1| 2959| 4.0|
 1| 4226| 4.0|
 1| 4878| 5.0|
 1| 5577| 5.0|
 1| 33794| 4.0|
 1| 54503| 3.5|
 1| 58559| 4.0|
 1| 59315| 5.0|
 1| 68358| 5.0|
 1| 69844| 5.0|
 1| 73017| 5.0|
 1| 81834| 5.0|
+------+-------+------+
only showing top 20 rows

In [7]:
df.describe().show()

+-------+-----------------+------------------+------------------+
summary| userId| movieId| rating|
+-------+-----------------+------------------+------------------+
 count| 26024289| 26024289| 26024289|
 mean| 135037.090248114|15849.109677040553|3.5280903543608817|
 stddev|78176.19722170352| 31085.25753139175|1.0654427636662234|
 min| 1| 1| 0.5|
 max| 270896| 176275| 5.0|
+-------+-----------------+------------------+------------------+

In [8]:
# create evaluation data
training, test = df.randomSplit([0.8,0.2])

In [9]:
# create model
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [10]:
# train model
model = als.fit(training)

In [11]:
# get predictions
predictions = model.transform(test)

In [12]:
# look into predictions
predictions.describe().show()

+-------+------------------+------------------+------------------+----------+
summary| userId| movieId| rating|prediction|
+-------+------------------+------------------+------------------+----------+
 count| 5207240| 5207240| 5207240| 5207240|
 mean|134965.45787480508|15833.952381107842|3.5274914350020357| NaN|
 stddev| 78168.15524261474| 31076.10042506936|1.0659462189038542| NaN|
 min| 1| 1| 0.5| -7.354435|
 max| 270896| 176273| 5.0| NaN|
+-------+------------------+------------------+------------------+----------+

In [13]:
# drop rows with non predictions
predictions = predictions.na.drop()

In [14]:
# again look into predictions
predictions.describe().show()

+-------+-----------------+------------------+------------------+-----------------+
summary| userId| movieId| rating| prediction|
+-------+-----------------+------------------+------------------+-----------------+
 count| 5203823| 5203823| 5203823| 5203823|
 mean|134968.3183219337|15784.588798658217|3.5276688888150116|3.409396140958235|
 stddev| 78167.2518136652| 30975.19748918725|1.0658286200866753|0.751196973232041|
 min| 1| 1| 0.5| -7.354435|
 max| 270896| 176249| 5.0| 15.084001|
+-------+-----------------+------------------+------------------+-----------------+

In [15]:
# evaluate
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating')
rmse = evaluator.evaluate(predictions)
rmse

Out[ 27 ]: 0.840721602694552